In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
import utils


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /home/ec2-user/anaconda3/envs/pytorch_p39/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from transformers import LlamaForCausalLM, LlamaTokenizer,GenerationConfig
from peft import (
    PeftModel,
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
)

### (1) Load Model

In [3]:
MICRO_BATCH_SIZE = 4
BATCH_SIZE = 64 #128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 100
LEARNING_RATE = 3e-4
CUTOFF_LEN = 256
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
VAL_SET_SIZE = 0.2
TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

DATA_PATH = "./customized_tunining_dataset.json"

In [4]:
device_map = "auto"

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)

### load model after fine tuned on alpaca datasets
model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [5]:
document_name = "standard chartered bank smart credit card"

In [6]:
def generate_prompt(data_item):
    
    input = data_item.get("input","")
    input = ""# we don't use input for fine tuning
    instruction = data_item.get("instruction","")
    output = data_item.get("output","")
    
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:{output}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request according to {document_name} facts.

### Instruction:
{instruction}

### Response:{output}"""

In [7]:
eval_generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

In [8]:
def evaluate(data_item):
    prompt = generate_prompt(data_item)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=eval_generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Response:", output.split("### Response:")[1].strip())

In [9]:
input = None
instruction = "What is the offer period for the 8% cashback offer on the Standard Chartered Smart Credit Card?"

In [10]:
data_item ={"instruction":instruction}

In [11]:
evaluate(data_item)

Response: The offer period for the 8% cashback offer on the Standard Chartered Smart Credit Card is 6 months.


In [12]:
# model = prepare_model_for_int8_training(model)

In [13]:
# config = LoraConfig(
#     r=LORA_R,
#     lora_alpha=LORA_ALPHA,
#     target_modules=TARGET_MODULES,
#     lora_dropout=LORA_DROPOUT,
#     bias="none",
#     task_type="CAUSAL_LM",
# )
# model = get_peft_model(model, config)
  # unk. we want this to be different from the eos token

### (2) Load Fine Tunning Data

In [6]:
data = load_dataset("json", data_files=DATA_PATH)

Found cached dataset json (/home/ec2-user/.cache/huggingface/datasets/json/default-cb7b34f9451b7d92/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
VAL_SET_SIZE = int(VAL_SET_SIZE*len(data['train']))

In [8]:
train_val = data["train"].train_test_split(
    test_size=VAL_SET_SIZE, shuffle=True, seed=42
)
train_data = train_val["train"]
val_data = train_val["test"]

len(train_data)/(len(train_data)+len(val_data))

Loading cached split indices for dataset at /home/ec2-user/.cache/huggingface/datasets/json/default-cb7b34f9451b7d92/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-7cb0978bc07b99dc.arrow and /home/ec2-user/.cache/huggingface/datasets/json/default-cb7b34f9451b7d92/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-0abd0f1249d4461f.arrow


0.803921568627451

In [9]:
def generate_prompt(instruction):
    template =  f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
            ### Instruction:
            {instruction}
            ### Response:"""
    return template 

In [10]:
tokenizer.pad_token_id = 0
def tokenize(prompt):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }


In [11]:
train_dataset = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x['instruction'])))
val_dataset = val_data.shuffle().map(lambda x: tokenize(generate_prompt(x['instruction'])))

Map:   0%|          | 0/123 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

### (3) Evaluate before further fine tunning

In [12]:
eval_generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

In [13]:
def evaluate(instruction):
    prompt = generate_prompt("instruction")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=eval_generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Response:", output.split("### Response:")[1].strip())

In [14]:
dataset  = utils.open_json(DATA_PATH)

In [20]:
import random

In [21]:
sample = random.choice(dataset)

In [22]:
sample

{'instruction': 'What is the CashBack offer for Klook purchases with the Smart Card?',
 'input': 'Answer questions base on below facts: These are the facts about standard chartered bank smart credit card: Smart Card 3-month Interest-free Instalment Remarks\n\n^For spending amounts that are not eligible to earn CashBack (e.g. bill payment transactions made through Standard Chartered Online Banking), the Bank will also deduct HKD1 CashBack for every HKD180 instalment amount applied. Where there is insufficient CashBack balance on the 360° Rewards Redemption Platform for deduction, a negative balance will be displayed. 8% CashBack at Klook\n\nFrom now till 31 March 2023, enjoy 8% CashBack at Klook with Smart Card. No registration and minimum spending requirement.',
 'output': 'From now till 31 March 2023, Smart Card holders can enjoy 8% CashBack on Klook purchases, without any registration or minimum spending requirement.'}

In [23]:
instruction = sample['instruction']

In [24]:
evaluate(instruction)

Response: The answer to the question is "yes".


###  (4) Training

In [25]:
num_step_per_epoch = len(train_data)

In [26]:
print(num_step_per_epoch)

123


In [27]:
EPOCHS = 100

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=num_step_per_epoch,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=20,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=num_step_per_epoch*10,
        save_steps=num_step_per_epoch*10,
        output_dir="lora-alpaca",
        save_total_limit=3,
        load_best_model_at_end=True,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [28]:
model.config.use_cache = False

In [29]:
old_state_dict = model.state_dict

model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())
).__get__(model, type(model))


In [30]:
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/datasets/formatting/formatting.py:100: RuntimeWarning: divide by zero encountered in remainder
  return table.fast_gather(key % table.num_rows)


IndexError: list index out of range

### (4) Evaluation 

In [46]:
data_item = {}
data_item['instruction']='Can I earn reward points for paying my bills through Internet or phone banking services with the Standard Chartered Bank Smart Credit Card?'
evaluate(data_item)

Response: Yes, you can earn reward points for paying your bills through Internet or phone banking services with the Standard Chartered Bank Smart Credit Card.

### Instruction:
Can I earn reward points for paying my bills through Internet or phone banking services with the Standard Chartered Bank Smart Credit Card?


### (5) Save Model

In [ ]:
# saving model
model_name_or_path = "alpaca-lora-7b"
peft_type = "PROMPT_TUNING"
task_type = "CAUSAL_LM"

peft_model_id = f"{model_name_or_path}_{peft_type}_{task_type}"

model.save_pretrained(peft_model_id)